In [ ]:
import serial

import time
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np

import re
import pandas as pd
import dask.dataframe as dd
import streamz.dataframe as sdf
# from streamz.dataframe import StreamingDataFrame

import threading, queue


In [ ]:
portAddr = "/dev/cu.usbmodem14201"
baud = 115200

In [ ]:
ser = serial.Serial(
    port=portAddr,
    baudrate=baud,
    bytesize=8,
    timeout=2,
    stopbits=serial.STOPBITS_ONE
)
ser.flushInput()



In [ ]:
plot_window = 20
y_var = np.array(np.zeros([plot_window]))

plt.ion()
fig, ax = plt.subplots()
line, = ax.plot(y_var)


In [ ]:
def process_serial_out(ascii_string):
    ascii_string += ','
    findall = re.findall(r'(?P<var>.*?):(?P<out>.*?),', msg)
    extracted = {k.strip(): v.strip() for k,v in findall}
    return extracted

In [ ]:

def arduino_worker(queue):
    while True:
        try:
            ser_bytes = ser.readline()
            try:
                decoded_bytes = ser_bytes.decode('ascii')
                row = process_serial_out(decoded_bytes)
                print(decoded_bytes)
                queue.put(row)
            except:
                print_exc()
                print("exception reading one line from arduino")
                continue
        except:
            print("Keyboard Interrupt")
            break

In [ ]:
def stride_boundary(row, this_stride):
#     just a dummy code. We need to implement the logic.
    if this_stride.qsize()>500:
        return True
    else:
        return False

In [ ]:


#worker to process one stride boundary, sort data into buckets
def stride_processor_worker(total_data):
    this_stride = queue.Queue()
    while True:
        #Infinitely 1. appends to this_stride, then when boundary is met,
        #appends that stride to total_data, reinitializes this_stride
        #and loops forever.
        try:
            row = queue.get()
            if stride_boundary(row, this_stride):
                #put away the current stride in total data as one stride
                total_data.put(this_stride)
                print("finished writing this one stride", this_stride.qsize()
                     )
                #reinitialize the stride queue
                this_stride = queue.Queue()
            else:
                this_stride.put(row)
        except:
            print("exception in stride processor worker")
            
    

In [ ]:
usb_reading_queue = queue.Queue()
total_data = queue.Queue()

In [ ]:

#  set up worker threads
arduino_thread = threading.Thread(target=arduino_worker, args=[usb_reading_queue], daemon=True)

In [ ]:
stride_thread = threading.Thread(target=stride_processor_worker, args=[total_data], daemon=True)

In [ ]:
# start
arduino_thread.start()
stride_thread.start()

In [ ]:
# end
arduino_thread.join()
stride_thread.join()

In [ ]:
ser_bytes = ser.readline()

In [ ]:
ser_bytes

In [ ]:
sdf = (source.map(pd.read_csv)                  # stream of Pandas dataframes
             .to_dataframe(example=...))        # logical streaming dataframe


In [ ]:

this_stride = []
full_data = []
data_chunk = []
while True:
#     if [chu]
    try:
        ser_bytes = ser.readline()
        try:
            decoded_bytes = ser_bytes.decode('ascii')
            row = process_serial_out(decoded_bytes)
            print(decoded_bytes)
            data_chunk.append(row)
        except:
            print("exception?")
            continue
#         with open("test_data.csv","a") as f:
#             writer = csv.writer(f,delimiter=",")
#             writer.writerow([time.time(),decoded_bytes])
#         y_var = np.append(y_var,decoded_bytes)
#         y_var = y_var[1:plot_window+1]
#         line.set_ydata(y_var)
#         ax.relim()
#         ax.autoscale_view()
#         fig.canvas.draw()
#         fig.canvas.flush_events()
    except:
        print("Keyboard Interrupt")
        break

In [ ]:
dummy = {'Time':'5948', 'Pitch': '3.65',
 'Yaw': '-119.85',
 'Roll': '179.54',
 'Accel_x': '-0.50',
 'Accel_y': '0.86',
 'Accel_z': '0.02',
 'Gyro_x': '-0.50',
 'Gyro_y': '0.86',
 'Gyro_z': '0.02',
 'Mag_x': '-0.50',
 'Magl_y': '0.86',
 'Mag_z': '0.02'}

In [ ]:
pd.DataFrame([dummy, dummy])

In [ ]:
df = pd.DataFrame(dummy, columns = [
    'Yaw'
,'Roll'
,'Accel_x'
,'Accel_y'
,'Accel_z'
,'Gyro_x'
,'Gyro_y'
,'Gyro_z'
,'Mag_x' 
,'Magl_y'
,'Mag_z'])


In [ ]:
pd.DataFrame.from_dict(dummy, orient='columns', dtype=None,typ='series')


In [ ]:
# serialPort = serial.Serial(port = portAddr, baudrate=115200, bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)

In [ ]:
with open('hexdatafile.dat', 'wb') as datafile:
    datafile.write(ser.read(10000))

In [ ]:
msg = "Pitch:3.65,Yaw:-119.85,Roll:179.54,Accel_x: -0.50,Accel_y: 0.86,Accel_z: 0.02,Gyro_x: -0.50,Gyro_y: 0.86,Gyro_z: 0.02,Mag_x: -0.50,Magl_y: 0.86,Mag_z: 0.02"
msg += ","

In [ ]:
findall = re.findall(r'(?P<var>.*?):(?P<out>.*?),', msg)

In [ ]:
extracted = {k.strip(): v.strip() for k,v in findall}

In [ ]:
extracted